In [17]:
import pandas as pd

## Data

In [18]:
data = pd.read_excel('DC_Channel.xlsx')
data

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,2020-01-24 21:44:16
1,facebook,facebook,2020-03-01 15:44:35
2,organic,facebook,2019-12-07 16:37:06
3,search,tv,2020-01-27 16:09:32
4,discovery,youtube,2019-10-05 12:57:07
...,...,...,...
227623,facebook,facebook_organic,2019-11-17 14:12:33
227624,google_organic,referral,2019-12-06 18:02:13
227625,facebook,facebook,2019-12-21 19:40:44
227626,facebook,facebook,2020-01-17 23:58:51


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227628 entries, 0 to 227627
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   attribution_technical  227628 non-null  object        
 1   attribution_survey     224984 non-null  object        
 2   account_creation_date  227628 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 5.2+ MB


In [20]:
channel_spend = pd.read_excel('5-Channel-Spend.xlsx',index_col=0)
channel_spend

,bing,display,facebook,search,youtube
Date,,,,,
201907,400,12,9000,13000,90
201908,900,20,13000,18500,180
201909,1000,13,10500,19000,100
201910,1100,19,17000,24000,130
201911,1300,29,23000,25000,550
201912,300,31,16000,38000,900
202001,2100,148,11000,41000,4360
202002,3700,94,14000,44000,2420


## Reserve only date and month

In [21]:
data['account_creation_date'] = data['account_creation_date'].dt.date
data['account_creation_date'] = (pd.DatetimeIndex(data['account_creation_date']).year)*100 + (pd.DatetimeIndex(data['account_creation_date']).month)
data['account_creation_date']

0         202001
1         202003
2         201912
3         202001
4         201910
           ...  
227623    201911
227624    201912
227625    201912
227626    202001
227627    201911
Name: account_creation_date, Length: 227628, dtype: int64

## Channel spend - attribution_technical

In [22]:
df_technical = pd.DataFrame(data, columns=['attribution_technical','account_creation_date'])
df_technical

,attribution_technical,account_creation_date
0,facebook,202001
1,facebook,202003
2,organic,201912
3,search,202001
4,discovery,201910
...,...,...
227623,facebook,201911
227624,google_organic,201912
227625,facebook,201912
227626,facebook,202001


In [23]:
channel_list = ['bing', 'display', 'facebook', 'search', 'youtube']
time_list = [201907,201908,201909,201910,201911,201912,202001,202002]

In [24]:
# Create empty dataframe
survey_result = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [25]:
for c in channel_list:
    count_list = []
    for t in time_list:
        count = len(df_technical[(df_technical['attribution_technical']==c) & (df_technical['account_creation_date']==t)])
        count_list.append(count)
    survey_result[c] = count_list

survey_result

,bing,display,facebook,search,youtube
201907,120,44,10011,1328,8
201908,71,74,9855,1939,15
201909,84,12,6129,1852,2
201910,86,50,9489,2333,11
201911,103,90,12995,2510,47
201912,28,112,9288,3770,76
202001,172,536,6289,4073,370
202002,303,341,7988,4300,205


In [26]:
spend_list = []
for c in channel_list:
    spend_list.append(channel_spend[c].sum())

spend_list

[10800, 366, 113500, 222500, 8730]

In [27]:
convert_list = []
for c in survey_result:
    convert_list.append(survey_result[c].sum())

convert_list

[967, 1259, 72044, 22105, 734]

### Average CAC

In [28]:
avg_cac_list = []
for i in range(0,5):
    if convert_list[i] == 0:
        avg_cac_list.append(0)
    else:
        avg_cac = round(spend_list[i]/convert_list[i],2)
        avg_cac_list.append(avg_cac)

avg_cac_list

[11.17, 0.29, 1.58, 10.07, 11.89]

### Marginal CAC

In [29]:
# Create empty dataframe
marg_cac = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [30]:
for c in channel_list:
    marg_cac_list = []
    for t in time_list:
        if survey_result[c][t] == 0:
            marg_cac_list.append(0)
        else:
            marg_cac_value = round(channel_spend[c][t]/survey_result[c][t],2)
            marg_cac_list.append(marg_cac_value)
    marg_cac[c] = marg_cac_list

marg_cac

,bing,display,facebook,search,youtube
201907,3.33,0.27,0.90,9.79,11.25
201908,12.68,0.27,1.32,9.54,12.00
201909,11.90,1.08,1.71,10.26,50.00
201910,12.79,0.38,1.79,10.29,11.82
201911,12.62,0.32,1.77,9.96,11.70
201912,10.71,0.28,1.72,10.08,11.84
202001,12.21,0.28,1.75,10.07,11.78
202002,12.21,0.28,1.75,10.23,11.80


In [32]:
marg_cac.loc['average_CAC'] = [11.17, 0.29, 1.58, 10.07, 11.89]

In [33]:
marg_cac

,bing,display,facebook,search,youtube
201907,3.33,0.27,0.90,9.79,11.25
201908,12.68,0.27,1.32,9.54,12.00
201909,11.90,1.08,1.71,10.26,50.00
201910,12.79,0.38,1.79,10.29,11.82
201911,12.62,0.32,1.77,9.96,11.70
201912,10.71,0.28,1.72,10.08,11.84
202001,12.21,0.28,1.75,10.07,11.78
202002,12.21,0.28,1.75,10.23,11.80
average_CAC,11.17,0.29,1.58,10.07,11.89


In [34]:
marg_cac.to_csv('cs-technical.csv')